In [ ]:
from utils.hole_tracker_V3 import HoleTracker
import numpy as np

Tracker = HoleTracker(
    10, 10, 10,
    tiebreak_method = "KDE-N4-BW0.1",
    update_method   = "AVG-N5",

    imu_hist_minlen = 4,
    imu_hist_maxlen = 20,
    
    thr_ddetect= 10.0

    )

Tracker.do_new_imu_logic( 0, np.array([[1, 0, 0]+[0]*3]))
Tracker.do_new_imu_logic( 1, np.array([[0, 1, 0]+[0]*3]))
Tracker.do_new_imu_logic( 2, np.array([[0, 0, 1]+[0]*3]))
Tracker.do_new_imu_logic( 3, np.array([[0, 1, 0]+[0]*3]))
Tracker.do_new_imu_logic( 4, np.array([[1, 0, 1]+[0]*3]))

Tracker.do_new_detection_logic(1.3, np.random.rand(5, 3))
Tracker.do_new_detection_logic(1.9, np.random.rand(5, 3))
Tracker.do_new_detection_logic(2.8, np.random.rand(5, 3))
Tracker.do_new_detection_logic(3.4, np.random.rand(5, 3))

# print(Tracker._flag_new_detection)
Tracker.do_new_imu_logic( 5, np.array([[1, 0, 1]+[0]*3]))
Tracker.do_new_imu_logic( 6, np.array([[1, 0, 1]+[0]*3]))

Tracker.do_new_detection_logic(5.6, np.random.rand(5, 3))

Tracker.do_new_imu_logic( 7, np.array([[1, 0, 1]+[0]*3]))
Tracker.do_new_imu_logic( 8, np.array([[1, 0, 1]+[0]*3]))

Tracker.do_new_detection_logic(7.46, np.random.rand(5, 3))

Tracker.do_new_imu_logic( 9, np.array([[1, 0, 1]+[0]*3]))
Tracker.do_new_imu_logic(10, np.array([[1, 0, 1]+[0]*3]))

Tracker.do_new_detection_logic(8.99, np.random.rand(5, 3))

print("\n")
print("imu_data", Tracker._imu_data)
print("estimate", Tracker._p_estimate)
print("detections",Tracker._p_detection)



[DEBUG]: initialized tracker!
[DEBUG]: processing new imu: doing nothing (only adding imu to memory)
[DEBUG]: processing new imu: doing nothing (only adding imu to memory)
[DEBUG]: processing new imu: doing nothing (only adding imu to memory)
[DEBUG]: processing new imu: doing nothing (only adding imu to memory)
[DEBUG]: processing new imu: doing nothing (only adding imu to memory)
[DEBUG]: tiebreaking the new detection! (might take mulitple add detects for 'kde' method)
[DEBUG]: tiebreaking the new detection! (might take mulitple add detects for 'kde' method)
[DEBUG]: tiebreaking the new detection! (might take mulitple add detects for 'kde' method)
[DEBUG]: tiebreaking the new detection! (might take mulitple add detects for 'kde' method)
[DEBUG]: got a tiebreak result as an initial detection!
populating stored detections with repeats
[DEBUG]: processing new imu: initializing estimate from detection!
[DEBUG]: processing new imu: updating estimate from estimate!
[DEBUG]: processing new 